In [1]:
import requests
import pandas as pd
from datetime import datetime
import time


# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('CllfFuOvEOBS09TAzabT9Q', 'UmOCgu0uvbPzHn6bJgPwZ5Yf6Rmx7w')

with open('pw.txt','r') as f:
    pw = f.read().rstrip('\n')
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'dspd1',
        'password': pw}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyAPI/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers['Authorization'] = f'bearer {TOKEN}'

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [2]:
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = pd.concat([df, pd.Series({
            'subreddit': post['data']['subreddit'],
            'subreddit_name_prefixed': post['data']['subreddit_name_prefixed'],
            'name': post['data']['name'],
            'author': post['data']['author'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']}).to_frame().T], ignore_index=True)

    return df


# 'approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 
# 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 
# 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 
# 'author_flair_background_color', 'subreddit_type', 'ups', 'total_awards_received', 'media_embed', 
# 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 
# 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 
# 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 
# 'author_flair_richtext', 'gildings', 'content_categories', 'is_self', 'mod_note', 'created', 'link_flair_type',
# 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html', 
# 'likes', 'suggested_sort', 'banned_at_utc', 'view_count', 'archived', 'no_follow', 'is_crosspostable', 'pinned', 
# 'over_18', 'all_awardings', 'awarders', 'media_only', 'can_gild', 'spoiler', 'locked', 'author_flair_text', 
# 'treatment_tags', 'visited', 'removed_by', 'num_reports', 'distinguished', 'subreddit_id', 'author_is_blocked', 
# 'mod_reason_by', 'removal_reason', 'link_flair_background_color', 'id', 'is_robot_indexable', 'report_reasons', 
# 'author', 'discussion_type', 'num_comments', 'send_replies', 'whitelist_status', 'contest_mode', 'mod_reports', 
# 'author_patreon_flair', 'author_flair_text_color', 'permalink', 'parent_whitelist_status', 'stickied', 'url', 
# 'subreddit_subscribers', 'created_utc', 'num_crossposts', 'media', 'is_video']

In [3]:
sub_reddits=['r/1P_LSD', 'r/2cb', 'r/4acodmt', 'r/adderall', 'r/afinil', 'r/ambien', 'r/AMT', 'r/Ayahuasca', 
'r/cocaine', 'r/cripplingalcoholism', 'r/DMT', 'r/DPH', 'r/dxm', 'r/fentanyl', 'r/ketamine', 'r/kratom', 
'r/LSA', 'r/LSD', 'r/MDMA', 'r/MemantineHCl', 'r/mescaline', 'r/meth', 'r/PCP', 
'r/phenibut', 'r/PsilocybinMushrooms', 'r/Salvia', 'r/shroomers', 'r/shrooms', 
'r/benzodiazepines', 'r/dissociatives', 'r/DissonautUniverse', 'r/gabagoodness', 
'r/microdosing', 'r/noids', 'r/opiates', 'r/Opioid_RCs', 'r/PsychedelicMessages', 
'r/Psychedelics', 'r/PsychedSubstance', 'r/Psychonaut', 'r/RationalPsychonaut', 'r/Stims', 
'r/treedibles', 'r/trees', 'r/tryptonaut', 'r/AnAnswerToHeal', 'r/AskDrugNerds', 'r/askdrugs', 'r/aves', 
'r/bestoferowid', 'r/CurrentlyTripping', 'r/druganalytics', 'r/druggardening', 'r/DrugNerds', 'r/Drugs', 
'r/DrugShowerThoughts', 'r/DrugsOver30', 'r/erowid', 'r/ObscureDrugs', 'r/ReagentTesting', 'r/researchchemicals', 
'r/samelevel', 'r/TheDrugClassroom', 'r/TheeHive', 'r/tripreports', 'r/TripSit', 'r/TripTales']

In [22]:
data = pd.read_csv('reddit.csv')
data.set_index(['subreddit', 'id'], inplace=True)
display(data)

subreddit_name_prefixed       name                author  \
subreddit id                                                                
1P_LSD    z7uqgd                r/1P_LSD  t3_z7uqgd           CandyCaynee   
          z7rp08                r/1P_LSD  t3_z7rp08          RedSupreme20   
          z7qszm                r/1P_LSD  t3_z7qszm          VeGaN_B0y_XL   
          z7csxh                r/1P_LSD  t3_z7csxh    Sea-Technician7704   
          z5na4c                r/1P_LSD  t3_z5na4c         WithMyRichard   
...                                  ...        ...                   ...   
TripTales 2pi17e             r/TripTales  t3_2pi17e         thatdarnmoose   
          2ph7b4             r/TripTales  t3_2ph7b4             heyguys09   
          2ph4zd             r/TripTales  t3_2ph4zd              aarpcard   
          2ph49m             r/TripTales  t3_2ph49m  I_DO_COCAINE_CCCYEAH   
          2pghin             r/TripTales  t3_2pghin             Onyxdeity   

                                                              title  \
subreddit id                                                          
1P_LSD    z7uqgd                                      It’s all over   
          z7rp08    Should I pay for tracking if I’m buying 1p lsd?   
          z7qszm  salute to all Gratefuldead fans, lovers of roc...   
          z7csxh      Trying 1p-lsd for the First Time This Weekend   
          z5na4c  How many days should you wait to trip after ta...   
...                                                             ...   
TripTales 2pi17e                              Transcending (Salvia)   
          2ph7b4                                        Never again   
          2ph4zd                            Reality is a Nasty Joke   
          2ph49m                            My first time on Salvia   
          2pghin                          Inaugural post: [Salvia.]   

                                                           selftext  \
subreddit id                                                          
1P_LSD    z7uqgd  With the raid on lizard, I think that all RC’s...   
          z7rp08                                                NaN   
          z7qszm                                                NaN   
          z7csxh  I’ve done shrooms 20 times or so my favorite d...   
          z5na4c  I have PTSD and anxiety issues, sometimes when...   
...                                                             ...   
TripTales 2pi17e  My first and only (so far) salvia trip led me ...   
          2ph7b4  Took salvia awhile back with some friends that...   
          2ph4zd  I took salvia for the first time exactly a wee...   
          2ph49m  I thought I was in the imagination of another ...   
          2pghin  At the bequest of our community (a whole two p...   

                 upvote_ratio    ups  downs  score link_flair_css_class  \
subreddit id                                                              
1P_LSD    z7uqgd          0.7    5.0    0.0    5.0                  NaN   
          z7rp08          1.0    5.0    0.0    5.0                  NaN   
          z7qszm         0.87   18.0    0.0   18.0                  NaN   
          z7csxh         0.72    3.0    0.0    3.0                  NaN   
          z5na4c         0.56    1.0    0.0    1.0                  NaN   
...                       ...    ...    ...    ...                  ...   
TripTales 2pi17e         0.83   13.0    0.0   13.0                  one   
          2ph7b4         0.92   24.0    0.0   24.0                  one   
          2ph4zd         0.94   34.0    0.0   34.0                  one   
          2ph49m         0.79   18.0    0.0   18.0                  one   
          2pghin         0.99  144.0    0.0  144.0                  one   

                           created_utc kind  
subreddit id                                 
1P_LSD    z7uqgd  2022-11-29T14:54:55Z   t3  
          z7rp08  2022-11-29T12:40:53Z   t3  
       

In [4]:
data = pd.DataFrame()
start = time.time()
for sub_reddit in sub_reddits:
    params = {'limit': 100}
    dig_deeper = True
    i=0
    while dig_deeper:
        i += 1
        dig_deeper = i<=100
        res = requests.get(f'https://oauth.reddit.com/{sub_reddit}/new',
                              headers=headers,
                              params=params)      
        if 'error' in res.json():
            if res.json()['error']==403:
                print(f'private subreddit {sub_reddit}')
            elif res.json()['error']==404:
                reason = res.json()['reason']
                print(f'not found {sub_reddit}, reason {reason}')
            dig_deeper = False
            continue
        

        new_df = df_from_response(res) #create new data frame
        if len(new_df)==0:
            continue
        # take the final row (oldest entry)
        row = new_df.iloc[len(new_df)-1]
                                                    # create fullname
                                            #         id = row['id']

                                            #         fullname = row['kind'] + '_' + row['id']
        # add/update fullname in params
        params['after'] = row['name']
#         res = requests.get(f'https://oauth.reddit.com/{sub_reddit}/comments/{id}', 
#                            params={'depth':50}, headers=headers)
#         comments = res.json()[1]['data']['children'][1]['data']['body']
 
        # append new_df to data
        data = pd.concat([data, new_df], ignore_index=True)
end = time.time()
print('time to scrap',end - start)


private subreddit r/cripplingalcoholism
not found r/DissonautUniverse, reason banned
private subreddit r/tryptonaut
time to scrap 2773.2158064842224


In [5]:
data.set_index(['subreddit', 'id'], inplace=True)


In [6]:
data.groupby('subreddit').count()['title'].max()

997

In [21]:
data.to_csv('reddit.csv')  
display(data)

subreddit_name_prefixed       name                author  \
subreddit id                                                                
1P_LSD    z7uqgd                r/1P_LSD  t3_z7uqgd           CandyCaynee   
          z7rp08                r/1P_LSD  t3_z7rp08          RedSupreme20   
          z7qszm                r/1P_LSD  t3_z7qszm          VeGaN_B0y_XL   
          z7csxh                r/1P_LSD  t3_z7csxh    Sea-Technician7704   
          z5na4c                r/1P_LSD  t3_z5na4c         WithMyRichard   
...                                  ...        ...                   ...   
TripTales 2pi17e             r/TripTales  t3_2pi17e         thatdarnmoose   
          2ph7b4             r/TripTales  t3_2ph7b4             heyguys09   
          2ph4zd             r/TripTales  t3_2ph4zd              aarpcard   
          2ph49m             r/TripTales  t3_2ph49m  I_DO_COCAINE_CCCYEAH   
          2pghin             r/TripTales  t3_2pghin             Onyxdeity   

                                                              title  \
subreddit id                                                          
1P_LSD    z7uqgd                                      It’s all over   
          z7rp08    Should I pay for tracking if I’m buying 1p lsd?   
          z7qszm  salute to all Gratefuldead fans, lovers of roc...   
          z7csxh      Trying 1p-lsd for the First Time This Weekend   
          z5na4c  How many days should you wait to trip after ta...   
...                                                             ...   
TripTales 2pi17e                              Transcending (Salvia)   
          2ph7b4                                        Never again   
          2ph4zd                            Reality is a Nasty Joke   
          2ph49m                            My first time on Salvia   
          2pghin                          Inaugural post: [Salvia.]   

                                                           selftext  \
subreddit id                                                          
1P_LSD    z7uqgd  With the raid on lizard, I think that all RC’s...   
          z7rp08                                                NaN   
          z7qszm                                                NaN   
          z7csxh  I’ve done shrooms 20 times or so my favorite d...   
          z5na4c  I have PTSD and anxiety issues, sometimes when...   
...                                                             ...   
TripTales 2pi17e  My first and only (so far) salvia trip led me ...   
          2ph7b4  Took salvia awhile back with some friends that...   
          2ph4zd  I took salvia for the first time exactly a wee...   
          2ph49m  I thought I was in the imagination of another ...   
          2pghin  At the bequest of our community (a whole two p...   

                 upvote_ratio    ups  downs  score link_flair_css_class  \
subreddit id                                                              
1P_LSD    z7uqgd          0.7    5.0    0.0    5.0                  NaN   
          z7rp08          1.0    5.0    0.0    5.0                  NaN   
          z7qszm         0.87   18.0    0.0   18.0                  NaN   
          z7csxh         0.72    3.0    0.0    3.0                  NaN   
          z5na4c         0.56    1.0    0.0    1.0                  NaN   
...                       ...    ...    ...    ...                  ...   
TripTales 2pi17e         0.83   13.0    0.0   13.0                  one   
          2ph7b4         0.92   24.0    0.0   24.0                  one   
          2ph4zd         0.94   34.0    0.0   34.0                  one   
          2ph49m         0.79   18.0    0.0   18.0                  one   
          2pghin         0.99  144.0    0.0  144.0                  one   

                           created_utc kind  
subreddit id                                 
1P_LSD    z7uqgd  2022-11-29T14:54:55Z   t3  
          z7rp08  2022-11-29T12:40:53Z   t3  
       

In [8]:
res = requests.get(f'https://oauth.reddit.com/r/opiates/comments/yu4a4s', params={'depth':50}, headers=headers)#comments/t3_yu4a4s/',


In [9]:
res = requests.get(f'https://oauth.reddit.com/r/opiates/new', params={'depth':50}, headers=headers)#comments/t3_yu4a4s/',


In [10]:
a = res.json()['data']['children']

In [11]:
a[1]['data']['author_fullname']

't2_cacvxnl0'

In [12]:
res = requests.get(f'https://oauth.reddit.com/{sub_reddit}/new',
                              headers=headers,
                              params=params)      


In [13]:
res.json()['data']['children'][1]

IndexError: list index out of range

In [ ]:
df_from_response(res)